In [22]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [29]:
le = LabelEncoder()

def load_data(file_name):
    data = pd.read_csv(file_name)
    data = data.drop(columns=['Unnamed: 0', 'x12'])
    data['x7'] = le.fit_transform(data['x7'])
    if file_name=='TrainOnMe.csv':
        data['y'] = le.fit_transform(data['y'])
        labels = data['y']
        features = data.drop(columns=['y'])
        return labels, features
    return data

labels, data = load_data('TrainOnMe.csv')
eval_data = load_data('EvaluateOnMe.csv')

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x13
0,198.87357,-1.56416,-105.28502,-1.05843,228.80634,-119.57104,0,-0.52242,-1.45859,11.59522,-0.96454,941.72532
1,199.52698,0.54804,-97.07504,-1.04948,229.47260,-120.75295,1,1.60107,-1.91971,7.66421,-3.49630,949.09736
2,201.17314,-0.22721,-98.97896,-1.07013,231.08207,-122.64781,3,0.60019,-0.19340,10.53295,-4.81560,956.37621
3,200.14047,0.01896,-108.08573,-1.04395,230.09569,-120.48585,4,-1.25252,1.36110,15.50931,-1.59705,946.65947
4,199.80402,-0.93272,-104.79682,-1.05184,229.74889,-121.84848,1,1.48921,-1.35437,11.07695,-0.67000,946.62167
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,197.89196,0.16117,-98.54640,-1.05951,227.82161,-118.02619,0,-1.77770,0.45873,15.04339,-1.85403,940.18662
4996,200.44175,1.01078,-101.50406,-1.05232,230.37693,-123.80861,4,-0.76962,0.61397,14.25643,0.22269,951.45674
4997,199.41478,0.24258,-98.74651,-1.06685,229.32961,-124.04039,1,-0.68400,-0.07620,12.38464,-0.75454,947.70064
4998,199.91353,-1.11320,-100.68991,-1.05011,229.85840,-124.98454,1,-2.47273,1.35400,10.78334,-7.86002,949.22267


0       1
1       1
2       0
3       1
4       2
       ..
4995    1
4996    0
4997    0
4998    1
4999    1
Name: y, Length: 5000, dtype: int64

In [30]:
# Choose classifier
model = RandomForestClassifier(random_state=42)

In [31]:
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

# Apply cross validation
scores = cross_val_score(model, data, labels, cv=rskf, scoring='accuracy')

print("Repeated Stratified Cross-Validation Scores:", scores)
print("Average Accuracy:", scores.mean())

Repeated Stratified Cross-Validation Scores: [0.701 0.671 0.7   0.711 0.682 0.682 0.698 0.711 0.675 0.72  0.697 0.684
 0.7   0.667 0.693 0.697 0.702 0.68  0.69  0.708 0.7   0.711 0.698 0.703
 0.664 0.688 0.695 0.662 0.705 0.698 0.688 0.694 0.707 0.699 0.694 0.69
 0.715 0.682 0.713 0.684 0.714 0.7   0.69  0.682 0.696 0.656 0.727 0.695
 0.706 0.691]
Average Accuracy: 0.6943200000000002


In [27]:
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats

# Define the parameter distribution
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2', 0.5],
    'bootstrap': [True, False]
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings sampled
    scoring='accuracy',  # Change this if needed (e.g., 'f1', 'roc_auc')
    cv=5,  # Cross-validation folds
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit the randomized search
random_search.fit(data, labels)

# Best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Cross-Validation Score:", random_search.best_score_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
130 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
108 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~

Best Parameters: {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 10, 'bootstrap': True}
Best Cross-Validation Score: 0.7021999999999998


In [21]:
# Train the model with the best parameters
best_model = random_search.best_estimator_




Tuned Model Accuracy: 0.70274
